In [2]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

# Algorithm 2 testing

In [3]:
# def Arnoldi(A, v, m): 
#     beta = norm(v)
#     v = v/beta # dimension of v is n x 1
#     H = sp.sparse.lil_matrix((m,m)) # dimension of H is m x m 
#     V = sp.sparse.lil_matrix((A.shape[0],m))
#     V[:,0] = v # each column of V is a vector v

#     for j in range(m):
#         print("j = ", j)
#         w = A @ v  
#         for i in range(j):
#             tmp = v.T @ w  
#             H[i,j] = tmp[0,0]
#             w = w - H[i,j]*v 
            
#         H[j,j-1] = norm(w) 

#         if H[j,j-1] == 0: 
#             print("Arnoldi breakdown")
#             m = j
#             v = 0
#             break
#         else:
#             if j < m-1:
#                 v = w/H[j,j-1]
#                 # for i in range(A.shape[0]):
#                 #     V[i,j+1] = v[i,0]
#                 V[:,j+1] = v

#     print(j, " iterations completed")
#     print("V = ", V.shape)
#     print("H = ", H.shape)
#     print("v = ", v.shape)
#     print("beta = ", beta)

#     return V, H, v, beta, j

In [18]:
#  review the restarted GMRES method (hereafter referred to as GMRES in short), which is a nonsymmetric Krylov subspace solver based on the Arnoldi decomposition procedure

# return the matrix V, the matrix Hp, the vector v, the scalar beta and the number of iterations j. Where V is a matrix of dimension n x m, Hp is an upper Hessenberg matrix of dimension (m+1) x m. Return also the m+1 vector of the basis V. Return h as the m+1,m element of Hp and the vector v as the m+1 element of the basis V.


def ArnoldiGMRES(A,v0,m):
    v = v0
    beta = norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        # print("j = ", j)
        w = A @ v  
        for i in range(j):
            tmp = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            H[i,j] = tmp[0,0] 
            w = w - H[i,j]*v 
            
        H[j+1,j] = norm(w)

        if H[j+1,j] == 0:
            print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    print(j, " iterations completed")
    print("V = ", V.shape)
    print("H = ", H.shape)
    print("v = ", v.shape)
    print("beta = ", beta)

    return V, H, v, beta, j  
 

Creating a small test case

In [14]:
m = 100
n = 110
A = sp.sparse.rand(n,n, density=0.1, format='lil')
v = sp.sparse.rand(n,1, density=0.1, format='lil')

In [19]:
ArnoldiGMRES(A,v,m)

99  iterations completed
V =  (110, 101)
H =  (101, 100)
v =  (110, 1)
beta =  2.1103247239370497


(<110x101 sparse matrix of type '<class 'numpy.float64'>'
 	with 10863 stored elements in List of Lists format>,
 <101x100 sparse matrix of type '<class 'numpy.float64'>'
 	with 4923 stored elements in List of Lists format>,
 <110x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 110 stored elements in Compressed Sparse Row format>,
 2.1103247239370497,
 99)